In [1]:
Sys.setenv("RETICULATE_PYTHON" = "/srv/conda/bin/python")

library(keras)
library(tensorflow)

In [2]:
tf_config()

TensorFlow v2.4.1 (/srv/conda/lib/python3.9/site-packages/tensorflow)
Python v3.9 (/srv/conda/bin/python)

In [3]:
tf_version()

[1] ‘2.4’

In [4]:
mnist <- dataset_mnist()
train_images <- mnist$train$x
train_labels <- mnist$train$y
test_images <- mnist$test$x
test_labels <- mnist$test$y

In [5]:
str(train_images)

 int [1:60000, 1:28, 1:28] 0 0 0 0 0 0 0 0 0 0 ...


In [6]:
str(train_labels)

 int [1:60000(1d)] 5 0 4 1 9 2 1 3 1 4 ...


In [7]:
network <- keras_model_sequential() %>% 
  layer_dense(units = 512, activation = "relu", input_shape = c(28 * 28)) %>% 
  layer_dense(units = 10, activation = "softmax")

In [8]:
network %>% compile(
  optimizer = "rmsprop",
  loss = "categorical_crossentropy",
  metrics = c("accuracy")
)

In [9]:
train_images <- array_reshape(train_images, c(60000, 28 * 28))
train_images <- train_images / 255

test_images <- array_reshape(test_images, c(10000, 28 * 28))
test_images <- test_images / 255

In [10]:
train_labels <- to_categorical(train_labels)
test_labels <- to_categorical(test_labels)

In [11]:
network %>% fit(train_images, train_labels, epochs = 5, batch_size = 128)

In [ ]:
metrics <- network %>% evaluate(test_images, test_labels, verbose = 0)
metrics